# PyMem3DG Tutorial 1 - The Very Basic
`Cuncheng Zhu, Christopher T. Lee`

This tutorial provides a brief example for setting up a membrane simulation using PyMem3DG. The extensive documentations is hosted at https://rangamanilabucsd.github.io/Mem3DG/.

In [2]:
import pymem3dg as dg

## Mesh initialization
PyMem3DG currently provide functions to generate simple meshes, as well as API to read $\texttt{.ply}$ files. For example, we could generate icosphere for closed membrane simualation and existing meshes based on parametrized ellipsoids. 

In [3]:
icoFace, icoVertex = dg.getIcosphere(1, 3)
oblate = "sample_meshes/oblate.ply"
prolate = "sample_meshes/prolate.ply"

## Instantiate the $\texttt{System}$:  $\texttt{Parameters}$ and  $\texttt{Options}$ 

Specification of the struct `Paramters` is largely user-defined. `Parameters` is the structured into sub-structs. For example, the parameters for bending of `Parameters p` is `p.bending`. Please refer to the documentation for detailed explanation of various entries. Below we shows the simulation setup for a homogeneous membrane undergoing osmotic shrinkage, which is a classic solution of the *shape equation* that results in the biconcave shape similar to the red blood cell. Because of the simplicity of the setup, we only specify essential physical quantities such as the bending rigidity, stretching modulus and bulk modulus, assuming the default  and omitting the majority of the parameters. 

The trajectory of the simulations will be solely determined by the initial condition (initial geometry) and parameters. In this tutorial, we will see how initial geometry affects the results. 

In [4]:
p = dg.Parameters()

p.bending.Kbc = 8.22e-5
p.tension.Ksg = 0.1
p.tension.At = 12.4866
p.osmotic.isPreferredVolume = True
p.osmotic.Kv = 0.02
p.osmotic.Vt = 0.7 * 3.14 * 4 / 3

## Integration and optimization

PyMem3DG provides various evolvers to integrate the membrane dynamics or find mechanical/chemical equalibrium as the energy minimizer. Please refer to the documentation for details and guidelines. For simplicity, the forward euler method, mathematically equivalent to steepest descent method for optimization is used to integrate the three systems. The following integration is pre-runned. Uncomment `integrate()` to rerun them.

We first to integrate the oblate shape and we expect to form biconcave shape at the equalibrium. To visualize the result using polyscope, please refer to the documentation and a dedicated tutorial for visualization.


In [5]:

h = 1 # time step
outputDir = "output/tutorial1/biconcave"
g = dg.System(inputMesh = oblate, parameters = p)
g.initialize()

fe = dg.Euler(system = g, characteristicTimeStep = h, 
              totalTime = 2000 * h, savePeriod = 200 * h, 
              tolerance = 1e-5, outputDirectory = outputDir)
fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
# fe.integrate() 


area_init = 12.4783
vol_init = 4.02477
Characteristic volume wrt to At = 4.14897


Getting ahead, we can use Polycope (https://polyscope.run/py/) to visualize the trajectory. The detail will be provide in Tutorial 4. PyMem3DG provides a wrapper function to inspect the trajectory,  

In [6]:
import pymem3dg.visual as dg_vis
import polyscope as ps
dg_vis.animate(outputDir+"/traj.nc", meanCurvature = True)
ps.show()

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 Metal - 76.3


GLFW emitted error: Cocoa: Failed to find service port for display
GLFW emitted error: Cocoa: Failed to find service port for display


Similarly, we could have the same procedure with initial condition of prolate shape.

In [7]:
outputDir="output/tutorial1/dumbbell"
g = dg.System(inputMesh = prolate, parameters = p)
g.initialize()
fe = dg.Euler(g, 1, 100000, 10000,
                      1e-5, outputDir)
fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
# fe.integrate()


area_init = 12.4777
vol_init = 3.80903
Characteristic volume wrt to At = 4.14897


We can also do this on an generated icosphere. However, because of the symmetry of a icosphere, it appears that we need large osmotic pressure to kick start the deformation. To change the set up of $\texttt{System}$, it is generally safer practice to reinstantiate a new $\texttt{System}$ object. However, because of the simplicity of current setup, the rest of the system does not have complex dependency on attribute changes. Therefore we directly modify the underlying attribute $K_v$ and increase it by fivefold.

In [8]:
outputDir="output/tutorial1/star"
import pymem3dg.util as dg_util
icoVertex = dg_util.sphericalHarmonicsPerturbation(icoVertex, 5, 6, 0.1)
g = dg.System(icoFace, icoVertex, p)
g.initialize()
fe = dg.Euler(g, 0.1, 50000, 1000, 0, outputDir)
fe.ifPrintToConsole = True
fe.ifOutputTrajFile = True
# fe.integrate()

area_init = 

/Users/cuncheng/Dev/Mem3DG/python_src/pymem3dg/util.py:42: RuntimeWarning: divide by zero encountered in true_divide
  phi = np.arctan(z/r) + np.pi / 2


12.6076
vol_init = 4.15656
Characteristic volume wrt to At = 4.14897


Notices that the simulation is not ran until the equilibrium but up to $T = 70000$. It is interrupted intentionally due to corrupted mesh quality.

Notice that initial icosphere does not have constant mean curvature due to the existence of irregular points. The initial deformation comforms to the discretization defect. With the high curvature deformation, mesh quality deteriorates. To cope with the problem, please proceed to the next tutorial, where we introduce mesh regularization and mutation for adaptive meshing.